In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
import time

In [2]:
import gc

gc.collect()

7

In [3]:
class EnsemblePursuit():
    def __init__(self):
        self.nr_of_ensembles=None
        self.current_v=np.zeros(1)
        self.current_u=np.zeros(1)
        self.U=np.zeros(1)
        self.V=np.zeros(1)
        
    def calculate_cost_delta(self):
        #print(np.dot(self.current_v.T,self.current_v))
        cost_delta=np.clip(self.current_v.T@self.X,0,None)/np.dot(self.current_v.T,self.current_v)[0]-self.lambd
        cost_delta=cost_delta.reshape(self.X.shape[1],1)
        return cost_delta
    
    def fit_one_assembly(self):
        #Have to figure out a good initialization
        choose_neuron_idx=np.random.randint(0,self.X.shape[1],1)
        self.current_u=np.zeros((self.X.shape[1],1))
        self.current_u[choose_neuron_idx,0]=1
        self.current_v=self.X[:,choose_neuron_idx]
        max_delta_cost=1000
        self.i=0
        while max_delta_cost>0:
            cost_delta=self.calculate_cost_delta()
            excl_ind=np.where(self.current_u.flatten()==1)[0]
            masked_cost_delta=np.ma.array(cost_delta,mask=self.current_u)
            max_delta_neuron=np.ma.argmax(masked_cost_delta,fill_value=-1000)
            #print(max_delta_neuron)
            max_delta_cost=cost_delta[max_delta_neuron]
            if max_delta_cost>0:
                self.current_v=(self.current_v+self.X[:,max_delta_neuron].reshape(self.X.shape[0],1))/2
                self.current_u[max_delta_neuron,0]=1
            self.i+=1
        self.U=np.hstack((self.U,self.current_u))
        self.V=np.vstack((self.V,self.current_v.T))
        
    #def plot_heatmaps(self):
        
        
    def fit_transform(self,X,lambd,n_ensembles=None):
        self.X=[]
        self.X=X
        #print(self.X.shape)
        self.lambd=lambd
        self.U=np.zeros((self.X.shape[1],1))
        self.V=np.zeros((1,self.X.shape[0]))
        self.nr_of_neurons=[]
        for iteration in range(0,n_ensembles):
            self.fit_one_assembly()
            self.nr_of_neurons.append(self.i)
            self.X=(self.X.T-self.current_u@self.current_v.T).T
            self.X[self.X<0]=0
        self.U=self.U[:,1:]
        self.V=self.V[1:,:]
        return (self.U@self.V).T, self.nr_of_neurons, self.U, self.V

In [4]:
X=sio.loadmat('/home/maria/Documents/EnsemblePursuit/data/natimg2800_M170717_MP034_2017-09-11.mat')['stim']['resp'][0][0]
print(X.shape)
X[X<0]=0
stim=sio.loadmat('/home/maria/Documents/EnsemblePursuit/data/natimg2800_M170717_MP034_2017-09-11.mat')['stim']['istim'][0][0]

(5880, 10103)


In [5]:
'''
def test_train_split(data,stim):
    unique, counts = np.unique(stim.flatten(), return_counts=True)
    count_dict=dict(zip(unique, counts))

    keys_with_enough_data=[]
    for key in count_dict.keys():
        if count_dict[key]==2:
            keys_with_enough_data.append(key)

    filtered_stims=np.isin(stim.flatten(),keys_with_enough_data)

    #Arrange data so that responses with the same stimulus are adjacent
    z=stim.flatten()[np.where(filtered_stims)[0]]
    sortd=np.argsort(z)
    istim=np.sort(z)
    X=data[filtered_stims,:]
    out=X[sortd,:].copy()

    x_train=out[::2,:]
    y_train=istim[::2]
    x_test=out[1::2,:]
    y_test=istim[1::2]
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test=test_train_split(X,stim)

def correlate_train_test(fit_x_train,x_test):
    corr_coeff_lst=[]
    for j in range(0,fit_x_train.shape[1]):
        for i in range(0,x_test.shape[1]):
            if i==j:
                corr_coeff_lst.append(np.corrcoef(fit_x_train[:,j].T,x_test[:,i].T)[0,1])
    corr_coeff_lst=np.nan_to_num(corr_coeff_lst)
    return corr_coeff_lst
'''

'\ndef test_train_split(data,stim):\n    unique, counts = np.unique(stim.flatten(), return_counts=True)\n    count_dict=dict(zip(unique, counts))\n\n    keys_with_enough_data=[]\n    for key in count_dict.keys():\n        if count_dict[key]==2:\n            keys_with_enough_data.append(key)\n\n    filtered_stims=np.isin(stim.flatten(),keys_with_enough_data)\n\n    #Arrange data so that responses with the same stimulus are adjacent\n    z=stim.flatten()[np.where(filtered_stims)[0]]\n    sortd=np.argsort(z)\n    istim=np.sort(z)\n    X=data[filtered_stims,:]\n    out=X[sortd,:].copy()\n\n    x_train=out[::2,:]\n    y_train=istim[::2]\n    x_test=out[1::2,:]\n    y_test=istim[1::2]\n    \n    return x_train, x_test, y_train, y_test\n\nx_train, x_test, y_train, y_test=test_train_split(X,stim)\n\ndef correlate_train_test(fit_x_train,x_test):\n    corr_coeff_lst=[]\n    for j in range(0,fit_x_train.shape[1]):\n        for i in range(0,x_test.shape[1]):\n            if i==j:\n                

In [6]:
np.random.seed(7)
ep=EnsemblePursuit()
s=time.time()
U_V,nr_of_neurons,U,V=ep.fit_transform(X,0.35,1000)
e=time.time()
print(e-s)
print(nr_of_neurons)
'''
corr_lst=correlate_train_test(fit_x_train,x_test)
corr_lst_sub=correlate_train_test(x_train-fit_x_train,x_test)
plt.hist(corr_lst)
plt.title('Correlations of neurons U@V vs test,'+str(n)+' , '+str(l))
plt.show()
plt.hist(corr_lst_sub)
plt.title('Correlations of neurons X-U@V vs test,'+str(n)+' , '+str(l))
plt.show()
plt.hist(n_of_neurons)
plt.title('Number of neurons in assemblies')
plt.show()'''

/home/maria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


1538.4936385154724
[6905, 845, 35, 198, 69, 73, 6, 2, 14, 35, 4, 62, 22, 25, 58, 72, 53, 105, 86, 64, 11, 63, 77, 11, 6, 31, 33, 9, 206, 158, 21, 8, 30, 182, 11, 7, 34, 18, 40, 116, 7, 15, 8, 84, 32, 3, 58, 137, 154, 1, 141, 19, 1, 10, 8, 25, 61, 22, 165, 66, 1, 1, 92, 1, 32, 53, 19, 64, 285, 2, 12, 24, 3, 1, 30, 1, 57, 72, 9, 9, 35, 1, 4, 37, 2, 6, 1, 51, 23, 17, 1, 1, 62, 10, 41, 5, 28, 86, 27, 7, 69, 8, 16, 26, 1, 45, 18, 6, 4, 40, 1, 80, 8, 47, 1, 5, 1, 104, 1, 14, 1, 1, 49, 14, 27, 25, 82, 34, 1, 19, 21, 17, 67, 1, 7, 1, 113, 86, 1, 48, 23, 29, 27, 28, 8, 42, 35, 1, 8, 4, 22, 35, 24, 36, 1, 44, 21, 46, 69, 18, 9, 31, 17, 6, 1, 20, 1, 6, 12, 1, 45, 9, 98, 1, 1, 9, 11, 20, 1, 1, 5, 3, 4, 12, 41, 1, 4, 23, 67, 4, 1, 4, 2, 2, 107, 5, 1, 117, 1, 74, 1, 25, 42, 33, 22, 2, 60, 12, 66, 2, 32, 125, 3, 27, 11, 12, 1, 2, 34, 52, 7, 9, 1, 23, 9, 36, 1, 1, 14, 61, 1, 1, 16, 22, 23, 1, 57, 72, 24, 2, 1, 1, 104, 34, 61, 1, 44, 4, 12, 2, 1, 15, 46, 45, 2, 57, 1, 130, 20, 42, 1, 1, 43, 46, 16, 10,

"\ncorr_lst=correlate_train_test(fit_x_train,x_test)\ncorr_lst_sub=correlate_train_test(x_train-fit_x_train,x_test)\nplt.hist(corr_lst)\nplt.title('Correlations of neurons U@V vs test,'+str(n)+' , '+str(l))\nplt.show()\nplt.hist(corr_lst_sub)\nplt.title('Correlations of neurons X-U@V vs test,'+str(n)+' , '+str(l))\nplt.show()\nplt.hist(n_of_neurons)\nplt.title('Number of neurons in assemblies')\nplt.show()"

In [7]:
#plt.hist(fit_x_train.flatten())